In [25]:

from openai import OpenAI
import mysql.connector
from pydantic import BaseModel, Field
import json

client = OpenAI(api_key='')

In [23]:
cod_base= client.files.create(
    file= open('codigos_unspsc.pdf', 'rb'),
    purpose= 'user_data'
)
cod_lic1= client.files.create(
    file= open('cod_lic1.pdf', 'rb'),
    purpose= 'user_data'
)
cod_lic2= client.files.create(
    file= open('cod_lic2.pdf', 'rb'),
    purpose= 'user_data'
)
cod_lic3= client.files.create(
    file= open('cod_lic3.pdf', 'rb'),
    purpose= 'user_data'
)

In [41]:
# Conexión a la base de datos:
class ConexionMySQL:
    def __init__(self, host, port, user, password, database):
        self.host = host
        self.port = port
        self.user = user
        self.password = password
        self.database = database

        # Aquí iría la lógica para conectarse a MySQL
        self.conexion = mysql.connector.connect(
            host=self.host,
            port=self.port,
            user=self.user,
            password=self.password,
            database=self.database
        )
        self.cursor = self.conexion.cursor()
        if self.conexion.is_connected():
            # Verificar la conexión
            print("Conexión exitosa a la base de datos MySQL")
        else:
            print("Error al conectar a la base de datos MySQL")

In [ ]:
# Pruebas con DB:
def main_mysql():
    # Crear una instancia de la clase ConexionMySQL
    db_mysql = ConexionMySQL(
        host="localhost",
        port=3306,
        user = "root",
        password = "admin",
        database = "db_insitel"
    )

    # Ejecutar una consulta a la base de datos
    db_mysql.cursor.execute("SELECT codigo_unspsc, descripcion FROM codigos_unspsc;")

    # Obtener los resultados
    resultados = db_mysql.cursor.fetchall()
    # Imprimir los resultados
    for fila in resultados:
        print(fila)

    # Cerrar la conexión
    db_mysql.cursor.close()
    db_mysql.conexion.close()
    print("Conexión cerrada")

main_mysql()

In [24]:
# Modelo de salida:
class CodigoComparado(BaseModel):
    cod_pedido: int = Field(description="Código UNSPSC solicitado en archivo de licitacion")
    cod_en_db: int = Field(description="Código UNSPSC en la base de datos (0 si no hay coincidencia)")
    cumple: bool = Field(description="True si el código está en la base de datos; False en caso contrario")

In [21]:
# Prompt propuesto
response = client.responses.create(
    model='gpt-4o-mini',
    input=[
        {
            #'role': 'system',
            #'content': [
            #    {'type': 'input_file', 'file_id': cod_base.id},
            #    {'type': 'input_text', 'text': (
            #        'Eres la empresa INSITEL S.A. y estos son los indicadores UNSPSC que manejamos actualmente, estos codigos son de 8 cifras, y se dividen cada dos digitos en segmento, familia, clase y producto, siempre interesa los 6 primeros digitos en conjunto para realizar comparaciones.'
            #   )}],
            'role': 'user',
            'content': [
                {'type': 'input_file', 'file_id': cod_base.id},
                {'type': 'input_file', 'file_id': cod_lic1.id},
                {'type': 'input_text', 'text': (
                    'Busca los códigos UNSPSC solicitados en el archivo de licitación dentro del archivo de datos de códigos UNSPSC registrados de INSITEL.'
                    'Para cada código solicitado, verifica si existe una coincidencia exacta en el archivo de datos de INSITEL de lo contrario devuelve 0.'
                    'Devuélveme el resultado como una lista JSON con el siguiente formato:\n\n'
                    '[\n'
                    '  {\n'
                    '    "cod_pedido": cod_unspsc_solicitado,\n'
                    '    "cod_en_db": cod_unspsc_db_insitel,\n'
                    '    "cumple": true\n'
                    '  },\n'
                    '  {\n'
                    '    "cod_pedido": cod_unspsc_solicitado,\n'
                    '    "cod_en_db": 0,\n'
                    '    "cumple": false\n'
                    '  }\n'
                    ']\n\n'
                    "No des ninguna explicación adicional, solo el arreglo JSON."
                )}
            ]
        }
    ]
)

In [22]:
salida = response.output_text.strip()

try:
    lista_dicts = json.loads(salida)
    codigos = [CodigoComparado(**item) for item in lista_dicts]
    for cod in codigos:
        print(cod)
except Exception as e:
    print("Error al convertir la salida en objetos CodigoComparado:", e)
    print("Respuesta del modelo:\n", salida)


Error al convertir la salida en objetos CodigoComparado: Expecting value: line 1 column 1 (char 0)
Respuesta del modelo:
 ```json
[
  {
    "cod_pedido": "41000000",
    "cod_en_db": "41000000",
    "cumple": true
  },
  {
    "cod_pedido": "41100000",
    "cod_en_db": "41100000",
    "cumple": true
  },
  {
    "cod_pedido": "41102600",
    "cod_en_db": "41102600",
    "cumple": true
  },
  {
    "cod_pedido": "43000000",
    "cod_en_db": "43000000",
    "cumple": true
  },
  {
    "cod_pedido": "43210000",
    "cod_en_db": "43210000",
    "cumple": true
  },
  {
    "cod_pedido": "43211700",
    "cod_en_db": "43211700",
    "cumple": true
  },
  {
    "cod_pedido": "46000000",
    "cod_en_db": "46000000",
    "cumple": true
  },
  {
    "cod_pedido": "46170000",
    "cod_en_db": "46170000",
    "cumple": true
  },
  {
    "cod_pedido": "46171600",
    "cod_en_db": "46171600",
    "cumple": true
  },
  {
    "cod_pedido": "32000000",
    "cod_en_db": "32000000",
    "cumple": true
  }